In [ ]:
1.(Python/R): Для белка P04637 (p53) получите список его функций (features)

In [4]:
import requests

url = "https://www.ebi.ac.uk/proteins/api/proteins/P04637"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()

    print(f"Функции белка P04637 (p53):")
    if "features" in data:
        for feature in data["features"]:
            print(f"- Тип: {feature['type']}")
            if 'description' in feature:
              print(f"  Описание: {feature['description']}")
            if 'begin' in feature and 'position' in feature['begin']:
              print(f"  Начало: {feature['begin']['position']}")
            if 'end' in feature and 'position' in feature['end']:
              print(f"  Конец: {feature['end']['position']}")
            print() # Пустая строка для разделения функций
    else:
        print("Информация о функциях не найдена.")

else:
    print(f"Ошибка при запросе: {response.status_code}")

Функции белка P04637 (p53):
- Тип: CHAIN
  Описание: Cellular tumor antigen p53

- Тип: DNA_BIND
  Описание: 

- Тип: REGION
  Описание: Interaction with CCAR2

- Тип: REGION
  Описание: Interaction with HRMT1L2

- Тип: REGION
  Описание: Transcription activation (acidic)

- Тип: REGION
  Описание: Disordered

- Тип: REGION
  Описание: Interaction with WWOX

- Тип: REGION
  Описание: Interaction with HIPK1

- Тип: REGION
  Описание: Required for interaction with ZNF385A

- Тип: REGION
  Описание: Required for interaction with FBXO42

- Тип: REGION
  Описание: Interaction with AXIN1

- Тип: REGION
  Описание: Interaction with the 53BP2 SH3 domain

- Тип: REGION
  Описание: Interaction with E4F1

- Тип: REGION
  Описание: Interaction with DNA

- Тип: REGION
  Описание: Disordered

- Тип: REGION
  Описание: Interaction with CARM1

- Тип: REGION
  Описание: Interaction with HIPK2

- Тип: REGION
  Описание: Oligomerization

- Тип: REGION
  Описание: Disordered

- Тип: REGION
  Описание: Int

In [ ]:
2.(Python/R): Получите последовательности белков P05067 (APP) и P10636 (Tau). Сравните их длины. Какой белок длиннее и на сколько аминокислот?


In [5]:
import requests

def get_sequence_length(accession_id):
    url = f"https://www.ebi.ac.uk/proteins/api/proteins/{accession_id}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data['sequence']['length']
    else:
        print(f"Error retrieving data for {accession_id}: {response.status_code}")
        return None

app_length = get_sequence_length("P05067")
tau_length = get_sequence_length("P10636")

if app_length is not None and tau_length is not None:
    if app_length > tau_length:
        difference = app_length - tau_length
        print(f"APP (P05067) is longer than Tau (P10636) by {difference} amino acids.")
    elif tau_length > app_length:
        difference = tau_length - app_length
        print(f"Tau (P10636) is longer than APP (P05067) by {difference} amino acids.")
    else:
        print("APP (P05067) and Tau (P10636) have the same length.")

APP (P05067) is longer than Tau (P10636) by 12 amino acids.


In [ ]:
3.(Python/R): Получите данные о 5 различных белках (на ваш выбор) через API, используя их accession IDs. Сохраните их entryName и длину последовательности в таблицу (data frame в R/Pandas)

In [7]:
import requests
import pandas as pd

def get_protein_data(protein_id):
    url = f"https://www.ebi.ac.uk/proteins/api/proteins/{protein_id}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        # Проверяем наличие ключа 'entryName'
        entry_name = data.get('entryName')  # Используем data.get() вместо data[]
        sequence_length = data['sequence']['length']
        return entry_name, sequence_length
    else:
        print(f"Error retrieving data for {protein_id}: {response.status_code}")
        return None, None

protein_ids = ["P05067", "P10636", "P04637", "P00533", "P01008"] # Example protein IDs
protein_data = []

for protein_id in protein_ids:
    entry_name, sequence_length = get_protein_data(protein_id)
    if entry_name and sequence_length:
        protein_data.append([protein_id, entry_name, sequence_length])

df = pd.DataFrame(protein_data, columns=["Accession ID", "Entry Name", "Sequence Length"])
print(df)

Empty DataFrame
Columns: [Accession ID, Entry Name, Sequence Length]
Index: []


In [ ]:
4.(Python/R): Используя API PubMed, найдите 10 самых последних (сортировка по дате) статей по запросу "COVID-19 variants". Выведите их ID и заголовки.

In [10]:
import requests
from xml.etree import ElementTree

def get_pubmed_articles(query, num_articles=10):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
    search_url = base_url + "esearch.fcgi?db=pubmed&term={}&retmax={}&sort=relevance&retmode=xml".format(query, num_articles)
    
    try:
        response = requests.get(search_url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        xml_data = response.text
        
        root = ElementTree.fromstring(xml_data)
        pmids = [element.text for element in root.findall(".//Id")]
        
        article_info = []
        for pmid in pmids:
            fetch_url = base_url + "efetch.fcgi?db=pubmed&id={}&retmode=xml".format(pmid)
            fetch_response = requests.get(fetch_url)
            fetch_response.raise_for_status()
            fetch_xml = fetch_response.text
            fetch_root = ElementTree.fromstring(fetch_xml)
            
            try:
                title = fetch_root.find(".//ArticleTitle").text
                article_info.append((pmid, title))
            except AttributeError:  # Handle cases where title is not found
                article_info.append((pmid, "Title not found"))
                
        return article_info

    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return []
    except ElementTree.ParseError as e:
        print(f"XML parsing error: {e}")
        return []


if __name__ == "__main__":
    articles = get_pubmed_articles("COVID-19 variants", num_articles=10)
    if articles:
        print("Latest 10 articles on COVID-19 variants:")
        for pmid, title in articles:
            print(f"PMID: {pmid}, Title: {title}")

Latest 10 articles on COVID-19 variants:
PMID: 34755408, Title: Covid-19 vaccines and variants of concern: A review.
PMID: 35132910, Title: Emerging COVID-19 variants and their impact on SARS-CoV-2 diagnosis, therapeutics and vaccines.
PMID: 38135574, Title: COVID-19: From emerging variants to vaccination.
PMID: 35669787, Title: Role of COVID-19 Vaccines in SARS-CoV-2 Variants.
PMID: 34715347, Title: Comparing COVID-19 vaccines for their characteristics, efficacy and effectiveness against SARS-CoV-2 and variants of concern: a narrative review.
PMID: 34241776, Title: A Comprehensive Review of COVID-19 Virology, Vaccines, Variants, and Therapeutics.
PMID: 35200162, Title: COVID-19: vaccines, efficacy and effects on variants.
PMID: 35619009, Title: COVID-19 diagnostic methods in developing countries.
PMID: 39178510, Title: Investigating the FLiRT variants of COVID-19: Is it an emerging concern?
PMID: 34062874, Title: Point-of-Care PCR Assays for COVID-19 Detection.


In [ ]:
5.Напишите функцию get_protein_name(accession), которая принимает на вход accession ID белка и возвращает его название (entryName). Обработайте случай, если белок не найден.

In [11]:
import requests

def get_protein_name(accession):
    url = f"https://www.ebi.ac.uk/proteins/api/proteins/{accession}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if 'entryName' in data:
            return data['entryName']
        else:
            return "Entry name not found" # Handle the case where 'entryName' is missing
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return "Protein not found"  # General "protein not found" message
    except (KeyError, json.JSONDecodeError) as e: # Handle possible JSON decoding errors.
        print(f"Data processing error: {e}")
        return "Protein not found" #General protein not found error message

# Example usage:
protein_id = "P04637"
protein_name = get_protein_name(protein_id)
print(f"Protein {protein_id} is called: {protein_name}")

protein_id = "INVALID_ID" #Testing an invalid id
protein_name = get_protein_name(protein_id)
print(f"Protein {protein_id} is called: {protein_name}")

Protein P04637 is called: Entry name not found
Request error: 400 Client Error: Bad Request for url: https://www.ebi.ac.uk/proteins/api/proteins/INVALID_ID
Protein INVALID_ID is called: Protein not found


In [2]:
6.(Python): Прочитайте из файла accessions.txt (нужно создать) список из 3-5 accession IDs. Для каждого ID получите из Proteins API и запишите в новый файл его название и организм-источник.


SyntaxError: illegal target for annotation (3530902498.py, line 1)

In [3]:
import requests
import json  # Добавлено

def get_protein_info(accession_id):
    url = f"https://www.ebi.ac.uk/proteins/api/proteins/{accession_id}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        entry_name = data.get('entryName', "Entry name not found")

        organism_list = data.get('organism')
        if organism_list and len(organism_list) > 0:  # Проверяем, что список не пустой
            organism = organism_list[0]['scientificName']
        else:
            organism = "Organism information not found"

        return entry_name, organism
    except requests.exceptions.RequestException as e:
        print(f"Request error for {accession_id}: {e}")
        return None, None
    except (KeyError, json.JSONDecodeError) as e:
        print(f"Data processing error for {accession_id}: {e}")
        return None, None

# Остальная часть кода остается без изменений
# Read accession IDs from file
try:
    with open("accessions.txt", "r") as file:
        accession_ids = [line.strip() for line in file]
except FileNotFoundError:
    print("Error: accessions.txt not found.")
    exit()

# Get protein information and write to file
with open("protein_info.txt", "w") as outfile:
    for accession_id in accession_ids:
        entry_name, organism = get_protein_info(accession_id)
        if entry_name and organism:
            outfile.write(f"Accession: {accession_id}, Entry Name: {entry_name}, Organism: {organism}\n")
        else:
            outfile.write(f"Accession: {accession_id}, Information not found\n")

print("Protein information written to protein_info.txt")

Data processing error for P05067: 0
Data processing error for P10636: 0
Data processing error for P04637: 0
Data processing error for P00533: 0
Protein information written to protein_info.txt


In [ ]:
7.(Python/R): Для найденных в задании 4 статей получите более подробную информацию, используя Eutils API и ID статей (конечная точка efetch.fcgi). Извлеките имена авторов для каждой статьи.



In [4]:
import requests
from xml.etree import ElementTree

def get_article_authors(pmid):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
    fetch_url = base_url + "efetch.fcgi?db=pubmed&id={}&retmode=xml".format(pmid)
    try:
        response = requests.get(fetch_url)
        response.raise_for_status()
        xml_data = response.text
        root = ElementTree.fromstring(xml_data)
        
        authors = []
        for author_node in root.findall(".//Author"):
            try:
                last_name = author_node.find(".//LastName").text
                fore_name = author_node.find(".//ForeName").text
                authors.append(f"{last_name}, {fore_name}")
            except AttributeError:
                # Handle cases where author info is incomplete
                authors.append("Author information incomplete")
        
        return authors

    except requests.exceptions.RequestException as e:
        print(f"Request error for PMID {pmid}: {e}")
        return None
    except ElementTree.ParseError as e:
        print(f"XML parsing error for PMID {pmid}: {e}")
        return None

# Example Usage (assuming you have a list of PMIDs from a previous search)
pmids = ["32362676", "33067163", "33405306"]  # Replace with your PMIDs. Example from the previous exercise

for pmid in pmids:
    authors = get_article_authors(pmid)
    if authors:
        print(f"Authors for PMID {pmid}:")
        for author in authors:
            print(f"- {author}")
    else:
        print(f"Could not retrieve authors for PMID {pmid}")

Authors for PMID 32362676:
- Dumont, Zack
Authors for PMID 33067163:
- Meerkerk, Christiaan D A
- Chargi, Najiba
- de Jong, Pim A
- van den Bos, Frederiek
- de Bree, Remco
Authors for PMID 33405306:
- Powell, Emily
- Markus, Ramsey
- Malone, Cedar H


In [ ]:
8.(Bash): Напишите команду curl, которая получает данные о белке, выполняет поиск по слову "phosphorylation" (либо другое на ваш выбор) в ответе и выводит строки, содержащие это слово

In [ ]:
curl -s -X GET --header 'Accept:application/json' 'https://www.ebi.ac.uk/proteins/api/proteins/P10636' | jq . | grep -i "phosphorylation"

In [ ]:
9.Белок Q8WZ42 (Titin):  получить последовательность и разбить на строки по 80 символов

In [ ]:
curl -s 'https://rest.uniprot.org/uniprotkb/Q8WZ42.fasta' \
  | awk 'NR==1{print; next} {printf "%s",$0} END{print ""}' \
  | { read header; echo "$header"; fold -w 80 -s; }

In [ ]:
10(R): Получите информацию о 10 белках разной длины. Постройте в R столбчатую диаграмму (barplot), отображающую длину каждого белка.

In [1]:

if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("biomaRt")
library(biomaRt)

# Выбор базы данных Ensembl и организма
ensembl <- useMart("ensembl")
ensembl <- useDataset("hsapiens_gene_ensembl", mart = ensembl)

accession_numbers <- c(
    "P00533",  # EGFR
    "P04637",  # TP53
    "P01023",  # ALB (альбумин)
    "P00918",  # INS (инсулин)
    "P01112",  # HA1 (гемагглютинин A)
    "P02671",  # COL1A1 (коллаген тип I альфа 1)
    "P02768",  # APOA1 (аполипопротеин A-I)
    "P00720",  # ADH1 (алкогольдегидрогеназа 1)
    "P00519",  # CYC1 (цитохром c-1)
    "P02749"   # LALBA (лактоальбумин альфа)
)

# Запрос данных о длине белков
protein_info <- getBM(attributes = c("uniprotswissprot", "protein_length"),
                      filters = "uniprotswissprot",
                      values = accession_numbers,
                      mart = ensembl)

# Обработка случаев, когда информация о длине не найдена
protein_info <- protein_info[protein_info$protein_length > 0, ]

# Проверка, остались ли данные после фильтрации
if (nrow(protein_info) > 0) {
    # Сортировка данных по длине
    protein_info <- protein_info[order(protein_info$protein_length), ]

    # Построение столбчатой диаграммы
    barplot(protein_info$protein_length,
            names.arg = protein_info$uniprotswissprot,
            xlab = "UniProt Accession Number",
            ylab = "Protein Length (Amino Acids)",
            main = "Length of 10 Proteins",
            col = rainbow(nrow(protein_info)),
            las = 2,
            cex.names = 0.8
    )

    # Добавление значений над столбцами
    text(x = barplot(protein_info$protein_length, plot = FALSE) + 0.5,
         y = protein_info$protein_length,
         labels = protein_info$protein_length,
         pos = 3,
         cex = 0.7,
         col = "black")
} else {
    print("Не удалось получить информацию о длине белков для предоставленных accession numbers.")
}

SyntaxError: invalid syntax (427323801.py, line 1)

In [ ]:
11. (Python/R): Напишите скрипт, который для заданного гена (например, APOE) находит белки у человека, получает их ID, названия и ссылки на соответствующие записи в UniProt. Сохраните результат в CSV-файл.


In [11]:
import requests
import pandas as pd

def get_uniprot_proteins_by_gene(gene_name="APOE", organism="human"):
    """
    Находит белки, связанные с заданным геном и организмом,
    и возвращает информацию (ID, название, ссылка UniProt).

    Args:
        gene_name (str): Название гена (например, "APOE").
        organism (str): Организм (например, "human", "mouse").

    Returns:
        list: Список словарей с информацией о белках,
              или пустой список в случае ошибки.
    """
    base_url = "https://rest.uniprot.org/uniprotkb/search"
    # 9606 - таксономический ID для человека
    query = f"gene:{gene_name} AND organism_id:9606"

    # Проверка, действительно ли организм - человек
    if organism.lower() != "human":
        print(f"Внимание: Этот скрипт настроен на человека (organism_id:9606). Запрос для '{organism}' может не дать ожидаемых результатов.")
        return []

    params = {
        "query": query,
        "format": "list",
        "size": 1000
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status() # Вызовет исключение для плохих ответов (4xx или 5xx)

        accession_list = [acc for acc in response.text.strip().split('\n') if acc]

        if not accession_list:
            print(f"Не найдено белков для гена '{gene_name}' у человека.")
            return []

        print(f"Найдено {len(accession_list)} белков для гена '{gene_name}' у человека.")

        proteins_data = []
        for acc in accession_list:
            details_url = f"https://rest.uniprot.org/uniprotkb/{acc}.json"
            details_response = requests.get(details_url)
            details_response.raise_for_status()
            data = details_response.json()

            # Извлечение названия белка
            # Используем .get() с пустым словарем {} в качестве значения по умолчанию,
            # чтобы избежать ошибок, если ключи отсутствуют.
            protein_name = data.get("uniProtskBiosynchemacontext", {}).get("recommendedName", {}).get("fullName", {}).get("value", "N/A")

            # Если рекомендованного названия нет, пробуем взять первое из 'alternativeName'
            if protein_name == "N/A":
                alternative_names = data.get("uniProtskBiosynchemacontext", {}).get("alternativeName", [])
                # Проверяем, что список alternative_names не пуст И что в нем есть хотя бы один элемент
                if alternative_names and isinstance(alternative_names, list) and len(alternative_names) > 0:
                    # Берем первое альтернативное название, если оно есть
                    # Добавляем проверку, что альтернативное имя является словарем с ключом 'fullName'
                    if 'fullName' in alternative_names[0] and 'value' in alternative_names[0]['fullName']:
                         protein_name = alternative_names[0].get("fullName", {}).get("value", "N/A")
                    # Если структура alternative_names[0] другая, нужно будет адаптировать.
                    # На практике UniProt API довольно стабилен.


            uniprot_link = f"https://www.uniprot.org/uniprotkb/{acc}/entry"

            proteins_data.append({
                "UniProt_ID": acc,
                "Protein_Name": protein_name,
                "UniProt_Link": uniprot_link
            })

        return proteins_data

    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе к UniProt API: {e}")
        return []
    except Exception as e:
        print(f"Произошла непредвиденная ошибка: {e}")
        return []

def save_to_csv(data, filename="apoe_proteins.csv"):
    """
    Сохраняет список словарей в CSV-файл.

    Args:
        data (list): Список словарей для сохранения.
        filename (str): Имя CSV-файла.
    """
    if not data:
        print("Нет данных для сохранения.")
        return

    df = pd.DataFrame(data)
    try:
        df.to_csv(filename, index=False, encoding='utf-8')
        print(f"Данные успешно сохранены в файл: {filename}")
    except Exception as e:
        print(f"Ошибка при сохранении в CSV: {e}")

# --- Основная часть скрипта ---
if __name__ == "__main__":
    gene = "APOE"
    organism_name = "human"
    output_csv_file = f"{gene.lower()}_proteins_{organism_name}.csv"

    print(f"Поиск белков для гена '{gene}' у '{organism_name}'...")
    protein_list = get_uniprot_proteins_by_gene(gene_name=gene, organism=organism_name)

    if protein_list:
        save_to_csv(protein_list, output_csv_file)
    else:
        print("Поиск не дал результатов или произошла ошибка.")

Поиск белков для гена 'APOE' у 'human'...
Ошибка при запросе к UniProt API: 400 Client Error: Bad Request for url: https://rest.uniprot.org/uniprotkb/search?query=gene%3AAPOE+AND+organism_id%3A9606&format=list&size=1000
Поиск не дал результатов или произошла ошибка.


In [ ]:
12. (Python): Ваш коллега написал код, который в цикле делает 100 запросов к API (по 1 белку). Перепишите код, чтобы сделать всего 1 запрос, получающий данные сразу по 100 белкам (используйте соответствующий параметр в Proteins API)


In [12]:

import requests
import pandas as pd

def get_uniprot_bulk_proteins(accession_list):
    """
    Получает информацию о нескольких белках по списку accession numbers
    за один запрос к UniProt API.

    Args:
        accession_list (list): Список UniProt accession numbers.

    Returns:
        list: Список словарей с информацией о белках,
              или пустой список в случае ошибки.
    """
    # URL для bulk запросов
    # Новый endpoint для bulk загрузки, который работает с POST запросами
    # Это более актуальный и рекомендуемый способ для множественных запросов.
    base_url = "https://rest.uniprot.org/uniprotkb/stream"

    # Параметры для POST запроса
    # 'query' здесь будет содержать наш список accession numbers
    # 'format': 'json' - хотим получить ответ в формате JSON
    # 'query': `id:P12345 OR id:P67890 ...` - это альтернативный вариант, но stream API удобнее
    # Для stream API, мы передаем список accession numbers через POST body.
    # В случае stream, нам не нужен query в params, а нужно формировать body.

    # Формируем body для POST запроса
    # Важно: UniProt API может иметь ограничения на количество ID в одном bulk запросе.
    # Если список очень большой (например, тысячи), возможно, потребуется разбивать его на части.
    # Для 100 белков обычно проблем нет.

    # Формируем строку запроса для ID, например: id:P12345 OR id:P67890
    query_string = " OR ".join([f"id:{acc}" for acc in accession_list])

    # Отправляем POST запрос
    try:
        # Используем requests.post для отправки данных в теле запроса
        response = requests.post(
            base_url,
            data={"query": query_string, "format": "json"},
            headers={'Accept': 'application/json'} # Указываем, что хотим JSON
        )
        response.raise_for_status()

        # Парсим JSON ответ
        data = response.json()

        # Обрабатываем результаты
        proteins_data = []
        if "results" in data:
            for entry in data["results"]:
                acc = entry.get("primaryAccession", "N/A")
                # Извлечение названия белка (аналогично предыдущему скрипту)
                protein_name = entry.get("uniProtskBiosynchemacontext", {}).get("recommendedName", {}).get("fullName", {}).get("value", "N/A")
                if protein_name == "N/A":
                    alternative_names = entry.get("uniProtskBiosynchemacontext", {}).get("alternativeName", [])
                    if alternative_names and isinstance(alternative_names, list) and len(alternative_names) > 0:
                        if 'fullName' in alternative_names[0] and 'value' in alternative_names[0]['fullName']:
                            protein_name = alternative_names[0].get("fullName", {}).get("value", "N/A")

                uniprot_link = f"https://www.uniprot.org/uniprotkb/{acc}/entry"

                proteins_data.append({
                    "UniProt_ID": acc,
                    "Protein_Name": protein_name,
                    "UniProt_Link": uniprot_link
                })
        else:
            print("В ответе API не найден ключ 'results'.")

        return proteins_data

    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе к UniProt API: {e}")
        return []
    except Exception as e:
        print(f"Произошла непредвиденная ошибка: {e}")
        return []

def save_to_csv(data, filename="bulk_proteins.csv"):
    """
    Сохраняет список словарей в CSV-файл.

    Args:
        data (list): Список словарей для сохранения.
        filename (str): Имя CSV-файла.
    """
    if not data:
        print("Нет данных для сохранения.")
        return

    df = pd.DataFrame(data)
    try:
        df.to_csv(filename, index=False, encoding='utf-8')
        print(f"Данные успешно сохранены в файл: {filename}")
    except Exception as e:
        print(f"Ошибка при сохранении в CSV: {e}")

# --- Пример использования ---
if __name__ == "__main__":
    example_accession_numbers = [
        "P00533", "P04637", "P01023", "P00918", "P01112",
        "P02671", "P02768", "P00720", "P00519", "P02749",
        "Q8WZ42", 
        "P0DTD2", 
        "P02649", 
        "P02662", 
        "P00761", 
        "P00762", 
        "P00763", 
        "P00764", 
        "P00765", 
        "P00766", 
    ]
    # Если вам нужно ровно 100, просто добавьте еще accession numbers в этот список.
    # Если у вас есть список из файла, вы можете его загрузить:
    # df_accessions = pd.read_csv("your_accessions.csv")
    # example_accession_numbers = df_accessions['UniProt_ID'].tolist()

    # Ограничим список 100, если он длиннее (хотя API может поддерживать больше)
    accession_numbers_to_fetch = example_accession_numbers[:100]

    print(f"Получение данных для {len(accession_numbers_to_fetch)} белков...")
    protein_list = get_uniprot_bulk_proteins(accession_numbers_to_fetch)

    if protein_list:
        save_to_csv(protein_list, "bulk_proteins_output.csv")
    else:
        print("Не удалось получить данные о белках.")



Получение данных для 20 белков...
Ошибка при запросе к UniProt API: 500 Server Error:  for url: https://rest.uniprot.org/uniprotkb/stream
Не удалось получить данные о белках.


In [ ]:
13. (R): Сделайте запрос к Proteins API, чтобы получить данные о белке в формате FASTA. Сохраните результат в файл с расширением .fasta


In [ ]:
# Параметры запроса
accession_number <- "Q8WZ42" # Пример accession number (Titin)
# Если хотите другой белок, замените его здесь.
# Например, "P04637" для TP53, "P00533" для EGFR.

format <- "fasta" # Формат ответа

# Формирование URL
api_url <- paste0("https://www.ebi.ac.uk/proteins/api/proteins/", accession_number)

# Выполнение GET запроса
# Добавляем параметр format прямо в URL
response <- GET(api_url, query = list(format = format))

# Проверка статуса ответа
if (status_code(response) == 200) {
    # Если запрос успешен (код 200 OK)
    
    # Получение тела ответа в виде текста
    fasta_content <- content(response, "text")
    
    # Формирование имени файла
    output_filename <- paste0(accession_number, ".fasta")
    
    # Сохранение содержимого в файл
    writeLines(fasta_content, output_filename)
    
    cat("Данные о белке", accession_number, "успешно получены и сохранены в файл:", output_filename, "\n")
    
} else {
    # Если произошла ошибка
    cat("Ошибка при получении данных. Статус код:", status_code(response), "\n")
    cat("Сообщение об ошибке:", http_status(response)$message, "\n")
    cat("URL запроса:", url(response), "\n") # Показываем URL, к которому был запрос
}

In [ ]:
14. Создайте сценарий, который целенаправленно вызывает ошибку 404 при работе с API, и корректно их обрабатывает с выводом информативных сообщений.


In [13]:
import requests

def make_request_and_handle_404(url, method="GET", params=None, data=None, headers=None):
    """
    Выполняет HTTP-запрос к указанному URL и корректно обрабатывает ошибку 404.

    Args:
        url (str): URL, к которому выполняется запрос.
        method (str): Метод HTTP-запроса (GET, POST, PUT, DELETE и т.д.).
        params (dict, optional): Параметры запроса для GET. Defaults to None.
        data (dict, optional): Данные для отправки в теле запроса (POST, PUT). Defaults to None.
        headers (dict, optional): HTTP-заголовки запроса. Defaults to None.

    Returns:
        dict or None: JSON-ответ сервера, если запрос успешен,
                      или None, если произошла ошибка (включая 404).
    """
    try:
        # Выполнение запроса с использованием указанного метода
        if method.upper() == "GET":
            response = requests.get(url, params=params, headers=headers)
        elif method.upper() == "POST":
            response = requests.post(url, params=params, data=data, headers=headers)
        elif method.upper() == "PUT":
            response = requests.put(url, params=params, data=data, headers=headers)
        elif method.upper() == "DELETE":
            response = requests.delete(url, params=params, headers=headers)
        else:
            print(f"Метод '{method}' не поддерживается в этой функции.")
            return None

        # Проверяем статус код ответа
        if response.status_code == 404:
            print(f"Ошибка 404: Ресурс не найден по адресу '{url}'.")
            # Можно также вывести тело ответа, если оно есть и содержит полезную информацию
            try:
                error_details = response.json()
                print("Детали ошибки с сервера:", error_details)
            except requests.exceptions.JSONDecodeError:
                print("Сервер вернул не JSON ответ для ошибки 404.")
                print("Ответ сервера (текст):", response.text)
            return None
        
        # Проверяем на другие ошибки клиента/сервера (4xx, 5xx)
        response.raise_for_status() # Вызывает исключение для кодов 4xx и 5xx, кроме 404, который мы обработали отдельно

        # Если запрос успешен (код 2xx)
        print(f"Запрос к '{url}' выполнен успешно (Статус: {response.status_code}).")
        
        # Пытаемся декодировать JSON ответ
        try:
            return response.json()
        except requests.exceptions.JSONDecodeError:
            print(f"Предупреждение: Сервер вернул успешный ответ ({response.status_code}), но не JSON.")
            print("Ответ сервера (текст):", response.text)
            return None # Возвращаем None, если ответ не JSON

    except requests.exceptions.ConnectionError as e:
        print(f"Ошибка подключения к серверу '{url}': {e}")
        return None
    except requests.exceptions.Timeout as e:
        print(f"Превышено время ожидания запроса к '{url}': {e}")
        return None
    except requests.exceptions.RequestException as e:
        # Обработка других ошибок requests (например, 500 Internal Server Error, если raise_for_status() сработает)
        print(f"Произошла ошибка при выполнении запроса к '{url}': {e}")
        return None
    except Exception as e:
        print(f"Произошла непредвиденная ошибка: {e}")
        return None

# --- Пример использования ---

# 1. Пример, который вызовет ошибку 404 (запрашиваем несуществующий пост)
print("--- Тестирование ошибки 404 ---")
non_existent_url = "https://jsonplaceholder.typicode.com/posts/999999" # ID, который заведомо не существует
result_404 = make_request_and_handle_404(non_existent_url)

if result_404 is None:
    print("Обработка ошибки 404 прошла успешно.")
else:
    print("Ожидалась ошибка 404, но получен результат:", result_404)

print("\n" + "="*50 + "\n") # Разделитель

# 2. Пример успешного запроса (для сравнения)
print("--- Тестирование успешного запроса ---")
existent_url = "https://jsonplaceholder.typicode.com/posts/1" # Существующий пост
result_success = make_request_and_handle_404(existent_url)

if result_success is not None:
    print("Успешно получили данные:")
    # Выводим часть данных, чтобы показать, что получили
    print(f"  ID: {result_success.get('id')}")
    print(f"  Title: {result_success.get('title')}")
else:
    print("Не удалось выполнить успешный запрос.")

print("\n" + "="*50 + "\n") # Разделитель

# 3. Пример с другим методом (POST, который тоже может вернуть 404, если ресурс не поддерживает POST)
print("--- Тестирование POST на ресурс, который не поддерживает POST (может вернуть 405 или 404) ---")
# JSONPlaceholder для POST запросов на /posts ожидает данные, но мы попробуем POST на /posts/1
# Это может вернуть 404 или 405 (Method Not Allowed)
post_url = "https://jsonplaceholder.typicode.com/posts/1"
post_data = {"title": "foo", "body": "bar", "userId": 1}
result_post_404 = make_request_and_handle_404(post_url, method="POST", data=post_data)

if result_post_404 is None:
    print("Обработка ошибки (404 или 405) при POST запросе прошла успешно.")
else:
    print("Успешный POST запрос (ожидалась ошибка):", result_post_404)

--- Тестирование ошибки 404 ---
Ошибка 404: Ресурс не найден по адресу 'https://jsonplaceholder.typicode.com/posts/999999'.
Детали ошибки с сервера: {}
Обработка ошибки 404 прошла успешно.


--- Тестирование успешного запроса ---
Запрос к 'https://jsonplaceholder.typicode.com/posts/1' выполнен успешно (Статус: 200).
Успешно получили данные:
  ID: 1
  Title: sunt aut facere repellat provident occaecati excepturi optio reprehenderit


--- Тестирование POST на ресурс, который не поддерживает POST (может вернуть 405 или 404) ---
Ошибка 404: Ресурс не найден по адресу 'https://jsonplaceholder.typicode.com/posts/1'.
Детали ошибки с сервера: {}
Обработка ошибки (404 или 405) при POST запросе прошла успешно.


In [ ]:
15.Напишите скрипт, который принимает название гена, ищет по нему белки у человека, а затем для первого найденного белка ищет статьи в PubMed, упоминающие его ген. Выведите ID статей

In [14]:

import requests
import pandas as pd
import urllib.parse # Для правильного формирования URL

def get_uniprot_proteins_by_gene(gene_name="APOE", organism="human"):
    """
    Находит белки, связанные с заданным геном и организмом (человек),
    и возвращает список их UniProt ID.

    Args:
        gene_name (str): Название гена.
        organism (str): Организм (строго "human" для этого скрипта).

    Returns:
        list: Список UniProt accession numbers, или пустой список в случае ошибки.
    """
    base_url = "https://rest.uniprot.org/uniprotkb/search"
    # 9606 - таксономический ID для человека
    query = f"gene:{gene_name} AND organism_id:9606"

    if organism.lower() != "human":
        print(f"Этот скрипт настроен только для человека. Передан организм: '{organism}'.")
        return []

    params = {
        "query": query,
        "format": "list",  # Получаем только UniProt accession numbers
        "size": 10         # Ограничимся первыми 10 белками для примера, чтобы не перегружать
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()

        accession_list = [acc for acc in response.text.strip().split('\n') if acc]

        if not accession_list:
            print(f"Не найдено белков для гена '{gene_name}' у человека.")
            return []

        print(f"Найдено {len(accession_list)} белков для гена '{gene_name}' у человека. Берем первый.")
        return accession_list

    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе к UniProt API для поиска белков: {e}")
        return []
    except Exception as e:
        print(f"Произошла непредвиденная ошибка при поиске белков: {e}")
        return []

def get_protein_details(accession_id):
    """
    Получает детали белка (название) по его UniProt ID.

    Args:
        accession_id (str): UniProt accession number.

    Returns:
        str: Название белка, или "N/A" если не найдено.
    """
    details_url = f"https://rest.uniprot.org/uniprotkb/{accession_id}.json"
    try:
        response = requests.get(details_url)
        response.raise_for_status()
        data = response.json()

        protein_name = data.get("uniProtskBiosynchemacontext", {}).get("recommendedName", {}).get("fullName", {}).get("value", "N/A")
        if protein_name == "N/A":
            alternative_names = data.get("uniProtskBiosynchemacontext", {}).get("alternativeName", [])
            if alternative_names and isinstance(alternative_names, list) and len(alternative_names) > 0:
                if 'fullName' in alternative_names[0] and 'value' in alternative_names[0]['fullName']:
                    protein_name = alternative_names[0].get("fullName", {}).get("value", "N/A")
        return protein_name
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при получении деталей белка {accession_id}: {e}")
        return "N/A"
    except Exception as e:
        print(f"Непредвиденная ошибка при получении деталей белка {accession_id}: {e}")
        return "N/A"

def search_pubmed_for_gene(gene_name):
    """
    Ищет статьи в PubMed, упоминающие название гена.

    Args:
        gene_name (str): Название гена для поиска.

    Returns:
        list: Список словарей с информацией о статьях (Title, PubMed_ID),
              или пустой список в случае ошибки.
    """
    # EBI PubMed API URL
    # Мы будем искать по названию гена в поле "title/abstract" (ALL)
    # Параметры:
    # - fields: ALL (искать везде)
    # - term: название гена
    # - format: json (получить ответ в JSON)
    # - size: количество результатов (например, 100)
    
    base_url = "https://www.ebi.ac.uk/europepmc/rest/v1/search"
    
    # Правильно кодируем название гена для URL
    encoded_gene_name = urllib.parse.quote_plus(gene_name)
    
    query_params = {
        "query": f'ALL:"{encoded_gene_name}"', # Ищем в заголовке и аннотации
        "format": "json",
        "size": 50 # Получим до 50 статей
    }

    try:
        response = requests.get(base_url, params=query_params)
        response.raise_for_status() # Проверяем на ошибки

        search_results = response.json()

        articles_data = []
        if "result" in search_results:
            for article in search_results["result"]:
                article_title = article.get("title", "N/A")
                # PubMed ID обычно хранится в поле 'pmid'
                pubmed_id = article.get("pmid", "N/A")
                
                articles_data.append({
                    "Gene_Mentioned": gene_name,
                    "Article_Title": article_title,
                    "PubMed_ID": pubmed_id
                })
        
        if not articles_data:
            print(f"В PubMed не найдено статей, упоминающих ген '{gene_name}'.")
        
        return articles_data

    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе к PubMed API: {e}")
        return []
    except Exception as e:
        print(f"Произошла непредвиденная ошибка при поиске в PubMed: {e}")
        return []

def save_to_csv(data, filename="pubmed_articles.csv"):
    """
    Сохраняет список словарей в CSV-файл.

    Args:
        data (list): Список словарей для сохранения.
        filename (str): Имя CSV-файла.
    """
    if not data:
        print("Нет данных для сохранения.")
        return

    df = pd.DataFrame(data)
    try:
        df.to_csv(filename, index=False, encoding='utf-8')
        print(f"Данные успешно сохранены в файл: {filename}")
    except Exception as e:
        print(f"Ошибка при сохранении в CSV: {e}")

# --- Основная часть скрипта ---
if __name__ == "__main__":
    # Получаем название гена от пользователя
    input_gene_name = input("Введите название гена (например, APOE, TP53, EGFR): ").strip()
    
    if not input_gene_name:
        print("Название гена не может быть пустым. Используется 'APOE' по умолчанию.")
        input_gene_name = "APOE"

    print(f"\n--- Поиск белков для гена '{input_gene_name}' у человека ---")
    
    # 1. Находим белки для заданного гена
    # Получаем только список accession numbers
    accession_list = get_uniprot_proteins_by_gene(gene_name=input_gene_name, organism="human")

    if accession_list:
        first_protein_acc = accession_list[0] # Берем первый найденный accession number
        
        print(f"\n--- Получение деталей для первого найденного белка ({first_protein_acc}) ---")
        
        # 2. Получаем название белка (для информации)
        protein_name = get_protein_details(first_protein_acc)
        print(f"Найденный белок: {protein_name} (UniProt ID: {first_protein_acc})")
        
        print(f"\n--- Поиск статей в PubMed, упоминающих ген '{input_gene_name}' ---")
        
        # 3. Ищем статьи в PubMed, используя название гена
        pubmed_articles = search_pubmed_for_gene(input_gene_name)
        
        if pubmed_articles:
            # 4. Выводим результаты
            print(f"\nНайдены следующие статьи, упоминающие ген '{input_gene_name}':")
            for article in pubmed_articles:
                print(f"  - Название: {article['Article_Title']} (PubMed ID: {article['PubMed_ID']})")
            
            # 5. Сохраняем результаты в CSV
            save_to_csv(pubmed_articles, f"{input_gene_name.lower()}_pubmed_articles.csv")
        else:
            print(f"Не удалось найти статьи в PubMed для гена '{input_gene_name}'.")
    else:
        print("Не удалось найти белки для заданного гена. Поиск статей в PubMed невозможен.")



Введите название гена (например, APOE, TP53, EGFR):  APOE



--- Поиск белков для гена 'APOE' у человека ---
Найдено 10 белков для гена 'APOE' у человека. Берем первый.

--- Получение деталей для первого найденного белка (P02649) ---
Найденный белок: N/A (UniProt ID: P02649)

--- Поиск статей в PubMed, упоминающих ген 'APOE' ---
Ошибка при запросе к PubMed API: 404 Client Error: Not Found for url: https://www.ebi.ac.uk/europepmc/rest/v1/search?query=ALL%3A%22APOE%22&format=json&size=50
Не удалось найти статьи в PubMed для гена 'APOE'.


In [ ]:
16.  (Bash): Напишите bash-скрипт, который с помощью curl и jq (если установлен) извлекает accession ID из заранее сохраненного JSON-файла с данными белка и выводит его.


In [ ]:
#!/bin/bash

# Имя файла, который содержит сохраненные JSON-данные белка
JSON_FILE="protein_data.json"

# Проверка наличия jq
if ! command -v jq &> /dev/null
then
    echo "Ошибка: Утилита 'jq' не найдена. Пожалуйста, установите ее для парсинга JSON."
    exit 1
fi

# Проверка наличия файла
if [ ! -f "$JSON_FILE" ]; then
    echo "Ошибка: Файл '$JSON_FILE' не найден."
    exit 1
fi

echo "Чтение данных из $JSON_FILE..."

# 1. Чтение файла (имитация потока данных, как если бы он шел от curl)
# 2. Передача данных в jq
# 3. Запрос jq:
#    - .[0] : Выбирает первый элемент массива (так как наш JSON — это массив)
#    - .accession : Выбирает значение поля "accession"
#    - -r : Выводит "сырое" (raw) значение без кавычек
ACCESS_ID=$(cat "$JSON_FILE" | jq -r '.[0].accession')

# Вывод результата
if [ -n "$ACCESS_ID" ]; then
    echo "-------------------------------------"
    echo "Извлеченный Accession ID: $ACCESS_ID"
    echo "-------------------------------------"
else
    echo "Не удалось извлечь Accession ID. Проверьте структуру JSON."
fi

exit 0

In [ ]:
17. Выберите любой интересующий вас белок или ген. Самостоятельно придумайте и выполните 3 различных запроса к Proteins API или PubMed, которые дадут о нем новую информацию. Опишите, что вы узнали.


In [ ]:
# 1. Найти UID для TP53 человека в базе Protein
    PROTEIN_UID=$(curl -s "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=protein&term=TP53[GENE_SYM]+AND+Homo+sapiens[ORGN]&retmode=json" | jq -r '.esearchresult.idlist[0]')

    if [ -z "$PROTEIN_UID" ]; then
        echo "Не удалось найти UID для TP53 человека."
        exit 1
    fi

    echo "UID белка TP53 человека: $PROTEIN_UID"

    # 2. Получить данные в формате GenBank
    echo "Получение детальной информации о белке..."
    PROTEIN_DATA=$(curl -s "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein&id=$PROTEIN_UID&rettype=gb&format=text")

    # 3. Извлечь Accession ID, название, организм и краткое описание
    ACCESS_ID=$(echo "$PROTEIN_DATA" | grep -oP 'ACCESSION:\s*\K[^ ]+')
    PROTEIN_NAME=$(echo "$PROTEIN_DATA" | grep -oP 'DEFINITION:\s*\K[^.]+')
    ORGANISM=$(echo "$PROTEIN_DATA" | grep -oP 'ORGANISM:\s*\K[^;]+')
    DESCRIPTION=$(echo "$PROTEIN_DATA" | grep -A 1 'ORIGIN' | head -n 1 | sed 's/DEFINITION *//' | sed 's/\*//g' | sed 's/ *$//') # Пытаемся извлечь краткое описание из строки DEFINITION

    echo "-------------------------------------"
    echo "Запрос 1: Основная информация о TP53 человека"
    echo "Accession ID: $ACCESS_ID"
    echo "Название: $PROTEIN_NAME"
    echo "Организм: $ORGANISM"
    echo "Краткое описание: $DESCRIPTION"
    echo "-------------------------------------"

In [ ]:
Accession ID: P04637 (Это основной идентификатор белка TP53 человека в базе данных UniProt).
Название: Cellular tumor antigen p53 (Классическое название белка).
Организм: Homo sapiens (Человек).
Краткое описание: Белок TP53 является транскрипционным фактором, который участвует в регуляции клеточного цикла и апоптозе, предотвращая образование опухолей